In [2]:
import os
import librosa
import librosa.display
import numpy
import numpy as np
import matplotlib.pyplot as plt

Create a list that contains start and stop seconds (=boundaries) of the cutted audio  


In [3]:
def save_boundaries(start, stop): 
    n=len(start)
    m=2
    l=[[0]*m for i in range(n)]
    for i in range(n) : 
        l[i][0] = start[i]
        l[i][1] = stop[i]
    return l

### Get files
This cell will fetch all wav files in the directories and subdirectories 

In [12]:
audio_dir = '/baie/travail/guyot/MRM/data/clean_corpus/2018/'

files_path = []
files_name = []
for path, subdirs, files in os.walk(audio_dir):
    for name in files: 
        if name.endswith(".wav"):
            files_path.append(os.path.join(path, name))
            files_name.append(name)
print("-", len(files_name), "files found in the directory", audio_dir)
print(files_name)

- 1 files found in the directory /baie/travail/guyot/MRM/data/clean_corpus/2018/
['2018-06-10_22-25.wav']


# Cut audio files + Save boundaries in text file + Mark the presence/absence of bull's sound

This cell consists of designing a system that, after cutting audio files and from annotation files, returns a binary decision for the presence / absence of bull's sound

In [13]:
features_dir= '/home/cmefteh/PFE/bulls_audio/audio_boundaries/'
s_dir = '/baie/travail/guyot/MRM/data/sliced_audio/audio_cut/'
bull_test = '/baie/travail/guyot/MRM/data/sliced_audio/test/'

ext1= '_boundaries.txt'
ext2= '.csv'

sr=44100
frame_length=10*sr
hop_length=10*sr

#samples_folder = os.path.join(s_dir, 'audio_splitted')

for file_path, file_name in zip(files_path, files_name):
    
    feature_file_name = features_dir + os.path.splitext(file_name)[0] + ext1
    test_file_name = bull_test + os.path.splitext(file_name)[0] + ext2
        
    #read the audio file
    print("reading the audio:", file_name)
    y, sr = librosa.load(file_path, sr)      
    
    start = np.arange(0, len(y), hop_length) / sr       
    stop = [x+(frame_length/float(sr)) for x in start]
    
    l = save_boundaries(start, stop) 
    
   
    
    #save all cutted audio boundaries in the directory "features_dir"
    np.savetxt(feature_file_name, l, fmt='%10d', delimiter=',')  
    
    #cut the audio file ______________________________
    
    # create a directory in "s_dir" having the same name of the original audio file where to save all the cutted audio from it  
    samples_folder = os.path.join(s_dir, os.path.splitext(file_name)[0]) 
    
    try:
        os.makedirs(samples_folder)
    except:
        pass
    
    for i in range(len(l)):
        
        #select every successive 10 sec in the original audio file
        x = y[int(l[i][0])*sr : int(l[i][1])*sr]       
        
        filename = os.path.join(samples_folder, os.path.splitext(file_name)[0] + "_" + str(int(start[i])).zfill(6) + '.wav')

        #save the frame selected as a wav file
        librosa.output.write_wav(filename, x, sr)     
        
    print("The audio: ",file_name, " was splitted successfully")
    
   
    
    #test the existence of the bulls in every cutted audio__________________________________
    #create a text file 
    nb_ligne=len(l)
    nb_colonne=2
    test_file=[[0]*nb_colonne for i in range(nb_ligne)]
    for i in range(nb_ligne) :
        
        #initialize all audios to 0 
        test_file[i][0] = os.path.splitext(file_name)[0] + "_" + str(int(start[i])).zfill(6) + '.wav'
         
    #open the annotation file corresponds to the already running audio
    with open ('{}.txt'.format(os.path.splitext(file_path)[0])) as f:
        txt_file = numpy.loadtxt(f)
        
        for i in range(len(l)):
            for j in range(len(txt_file)) :
                #length of bull  
                len_bull = txt_file[j][1] - txt_file[j][0]
                #take the bulls that are longer than two seconds 
                if (len_bull >= 2) :
                    if (start[i]<=txt_file[j][0]<=stop[i]) or (start[i]<=txt_file[j][1]<=stop[i]):
                    #give "1" to the audio if it contains a bull
                    #if not, leave it to "0"
                        test_file[i][1]="1"     
                                    
        
        #save the new modification after testing bulls existence
        np.savetxt(test_file_name, test_file, fmt='%s')   
        print("the presence/absence of bull's sound was marked successfully")
        print("__________________________") 

reading the audio: 2018-06-10_22-25.wav
The audio:  2018-06-10_22-25.wav  was splitted successfully
the presence/absence of bull's sound was marked successfully
__________________________
